In [2]:
import sys
import os

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("Running in Google Colab")
    if not os.path.exists("reliable-echo-segment"):
        !git clone https://github.com/khenm/reliable-echo-segment.git
    os.chdir("/content/reliable-echo-segment")
    sys.path.append("/content/reliable-echo-segment")
    %pip install -r requirements.txt -qq
else:
    print("Running locally")
    # Move up to project root if we are in notebooks directory
    if os.getcwd().endswith("notebooks"):
        os.chdir("..")
    
    project_root = os.getcwd()
    if project_root not in sys.path:
        sys.path.append(project_root)
    print(f"Working directory: {project_root}")

Running in Google Colab
Cloning into 'reliable-echo-segment'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 38 (delta 3), reused 35 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 18.85 KiB | 18.85 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [3]:
import yaml
import torch
import logging
import os
from src.utils.util_ import seed_everything, get_device
from src.dataset import get_dataloaders
from src.models.model import get_model
from src.trainer import Trainer
from src.utils.logging import get_logger

# 1. Create output directories
os.makedirs("checkpoints", exist_ok=True)
os.makedirs("runs", exist_ok=True)

# 2. Config Setup
config_path = "configs/config.yaml"
with open(config_path, 'r') as f:
    cfg = yaml.safe_load(f)

# Override paths to use new directories
cfg['training']['ckpt_save_path'] = os.path.join("checkpoints", "best_camus.pt")
cfg['training']['test_metrics_csv'] = os.path.join("runs", "camus_test_metrics.csv")

# 3. Logging Setup
# Add file handler to save logs to runs/train.log
logger = get_logger()
file_handler = logging.FileHandler(os.path.join("runs", "train.log"))
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

logger.info("Starting notebook run...")

# 4. Initialization
seed_everything(cfg['training']['seed'])
device = get_device()
logger.info(f"Using device: {device}")

# Data
logger.info("Initializing DataLoaders...")
loaders = get_dataloaders(cfg)

# Model
logger.info("Initializing Model...")
model = get_model(cfg, device)
logger.info(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

# 5. Training
trainer = Trainer(model, loaders, cfg, device)
trainer.train()

# 6. Evaluation
logger.info("Starting Evaluation...")
trainer.evaluate_test()
logger.info("Run finished.")

2025-12-19 04:14:50,418 - CAMUS - INFO - Starting notebook run...


INFO:CAMUS:Starting notebook run...


2025-12-19 04:14:50,427 - CAMUS - INFO - Using device: cpu


INFO:CAMUS:Using device: cpu


2025-12-19 04:14:50,429 - CAMUS - INFO - Initializing DataLoaders...


INFO:CAMUS:Initializing DataLoaders...


FileNotFoundError: [Errno 2] No such file or directory: '/Users/khenm/Materials/Datasets/CAMUS_public/database_split/subgroup_training.txt'